# Recommendation System

In this example we are creating a movie recommendation system by extractung feature vectors using PaddlePaddle, importing movie vectors into Milvus, and then searching in Milvus and Redis.

## Data
In this project, we use [MovieLens 1M](https://grouplens.org/datasets/movielens/1m/). This dataset contains 1,000,209 anonymous ratings of approximately 3,900 movies made by 6,040 MovieLens users. 

We use the following files:
- movies.dat: Contains movie information.
- movie_vectors.txt: Contains movie vectors that can be imported to Milvus easily.

File structure:

 - movMovieID::Title::Genres   

     - Titles are identical to titles provided by the IMDB (includingyear of release)
 
     - Genres are pipe-separated

     - Some MovieIDs do not correspond to a movie due to accidental duplicate entries and/or test entries
 
    - Movies are mostly entered by hand, so errors and inconsistencies may exist



## Requirements

Due to package constraints, this notebook needs to be run using Python 3.6/3.7 . It is recommended that you use a virtual enviroment like Conda, instructions for installing Conda can be found [here](https://conda.io/projects/conda/en/latest/user-guide/install/index.html). 

Currently, there is a dirty workaround that you can use. When installing the `requirements.txt`, pip will fail to install`sentencepiece`. If you rerun the notebook after the install fails and avoid redownloading the packages, the rest of the notebook should run without any hiccups.

|  Packages |  Servers |
| --------------- | -------------- |
| pymilvus        | milvus-1.1.0   |
| redis           | redis          |
| paddle_serving_app |
| paddlepaddle |


We have included a requirements.txt file in order to easily satisfy the required packages. 

## Up and Running

### Installing Packages
Install the required python packages with `requirements.txt`. If using Python 3.8, look at workaround in the Requirements section.

In [7]:
! pip install -r requirements.txt

  Using cached paddle_serving_app-0.3.1-py3-none-any.whl (52 kB)
  Using cached Shapely-1.7.1-cp38-cp38-macosx_10_9_x86_64.whl (1.0 MB)
  Using cached sentencepiece-0.1.83.tar.gz (497 kB)
    ERROR: Command errored out with exit status 1:
     command: /Users/filiphaltmayer/opt/miniconda3/envs/coinbase_demo/bin/python -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/t1/4dsdtlv12r7cpxx9hkz33b840000gn/T/pip-install-pj4cikfj/sentencepiece_83891e74c0de41d08e7b24394a6852ce/setup.py'"'"'; __file__='"'"'/private/var/folders/t1/4dsdtlv12r7cpxx9hkz33b840000gn/T/pip-install-pj4cikfj/sentencepiece_83891e74c0de41d08e7b24394a6852ce/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /private/var/folders/t1/4dsdtlv12r7cpxx9hkz

### Starting Milvus Server

This demo uses Milvus 1.1.0, please refer to the [Install Milvus](https://milvus.io/docs/v1.1.0/install_milvus.md) guide to learn how to use this docker container. For this example we wont be mapping any local volumes. 

In [12]:
!docker run -d --name milvus_cpu_1.1.0 \
-p 19530:19530 \
-p 19121:19121 \
milvusdb/milvus:1.1.0-cpu-d050721-5e559c

c24202156db5ee4dbe5a133d103dcd315313cbfb3e0c44bf26f34eaff8b50864


### Starting Redis Server
We are using Redis as a metadata storage service. Code can easily be modified to use a python dictionary, but that usually does not work in any use case outside of quick examples. We need a metadata storage service in order to be able to be able to map between embeddings and the corresponding data.

In [13]:
!docker run  --name redis -d -p 6379:6379 redis

c6593309b1ef1998e32b0fd66174e3e93fe360800c12cfb8ab6c4797ff3a384b


### Confirm Running Servers

In [17]:
! docker logs milvus_cpu_1.1.0


    __  _________ _   ____  ______    
   /  |/  /  _/ /| | / / / / / __/    
  / /|_/ // // /_| |/ / /_/ /\ \    
 /_/  /_/___/____/___/\____/___/     

Welcome to use Milvus!
Milvus Release version: v1.1.0, built at 2021-05-06 14:50.43, with OpenBLAS library.
You are using Milvus CPU edition
Last commit id: 5e559cd7918297bcdb55985b80567cb6278074dd

Loading configuration from: /var/lib/milvus/conf/server_config.yaml
WARNNING: You are using SQLite as the meta data management, which can't be used in production. Please change it to MySQL!
Supported CPU instruction sets: avx2, sse4_2
FAISS hook AVX2
Milvus server started successfully!


In [18]:
! docker logs redis

1:C 18 May 2021 19:36:22.376 # oO0OoO0OoO0Oo Redis is starting oO0OoO0OoO0Oo
1:C 18 May 2021 19:36:22.376 # Redis version=6.2.1, bits=64, commit=00000000, modified=0, pid=1, just started
1:C 18 May 2021 19:36:22.377 # Warning: no config file specified, using the default config. In order to specify a config file use redis-server /path/to/redis.conf
1:M 18 May 2021 19:36:22.379 * monotonic clock: POSIX clock_gettime
1:M 18 May 2021 19:36:22.381 * Running mode=standalone, port=6379.
1:M 18 May 2021 19:36:22.381 # Server initialized
1:M 18 May 2021 19:36:22.382 * Ready to accept connections


### Downloading Pretrained Models
This PaddlePaddle model is used to transform user information into vectors.

In [5]:
!wget https://paddlerec.bj.bcebos.com/aistudio/user_vector.tar.gz --no-check-certificate
!mkdir movie_recommender/user_vector_model
!tar xf user_vector.tar.gz -C movie_recommender/user_vector_model/
!rm user_vector.tar.gz

--2021-05-17 16:31:35--  https://paddlerec.bj.bcebos.com/aistudio/user_vector.tar.gz
Resolving paddlerec.bj.bcebos.com (paddlerec.bj.bcebos.com)... 103.235.46.61
Connecting to paddlerec.bj.bcebos.com (paddlerec.bj.bcebos.com)|103.235.46.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33650924 (32M) [application/x-gzip]
Saving to: ‘user_vector.tar.gz’

user_vector.tar.gz  100%[===================>]  32.09M  5.40MB/s    in 17s     

2021-05-17 16:31:54 (1.84 MB/s) - ‘user_vector.tar.gz’ saved [33650924/33650924]

mkdir: movie_recommender/user_vector_model: File exists


### Downloading Data

In [7]:
# Download movie information
!wget -P movie_recommender https://paddlerec.bj.bcebos.com/aistudio/movies.dat --no-check-certificate
# Download movie vecotrs
!wget -P movie_recommender https://paddlerec.bj.bcebos.com/aistudio/movie_vectors.txt --no-check-certificate

--2021-05-17 16:20:24--  https://paddlerec.bj.bcebos.com/aistudio/movies.dat
Resolving paddlerec.bj.bcebos.com (paddlerec.bj.bcebos.com)... 103.235.46.61
Connecting to paddlerec.bj.bcebos.com (paddlerec.bj.bcebos.com)|103.235.46.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 171308 (167K) [application/octet-stream]
Saving to: ‘movie_recommender/movies.dat’

movies.dat          100%[===================>] 167.29K  37.1KB/s    in 4.5s    

2021-05-17 16:20:31 (37.1 KB/s) - ‘movie_recommender/movies.dat’ saved [171308/171308]

--2021-05-17 16:20:31--  https://paddlerec.bj.bcebos.com/aistudio/movie_vectors.txt
Resolving paddlerec.bj.bcebos.com (paddlerec.bj.bcebos.com)... 103.235.46.61
Connecting to paddlerec.bj.bcebos.com (paddlerec.bj.bcebos.com)|103.235.46.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1095505 (1.0M) [text/plain]
Saving to: ‘movie_recommender/movie_vectors.txt’

movie_vectors.txt   100%[===================>]   1

## Code Overview


### Importing Movies into Milvus

#### 1. Connectings to Milvus and Redis
Both servers are running as Docker containers on the localhost with their corresponding default ports.

In [15]:
from milvus import Milvus, IndexType, MetricType, Status
import redis

milv = Milvus(host = '127.0.0.1', port = 19530)
r = redis.StrictRedis(host="127.0.0.1", port=6379) 

#### 2. Loading Movies into Redis
We begin by loading all the movie files into redis. 

In [16]:
import json
import codecs

#1::Toy Story (1995)::Animation|Children's|Comedy
def process_movie(lines, redis_cli):
    for line in lines:
        if len(line.strip()) == 0:
            continue
        tmp = line.strip().split("::")
        movie_id = tmp[0]
        title = tmp[1]
        genre_group = tmp[2]
        tmp = genre_group.strip().split("|")
        genre = tmp
        movie_info = {"movie_id" : movie_id,
                "title" : title,
                "genre" : genre
                }
        redis_cli.set("{}##movie_info".format(movie_id), json.dumps(movie_info))
        
with codecs.open("movie_recommender/movies.dat", "r",encoding='utf-8',errors='ignore') as f:
        lines = f.readlines()
        process_movie(lines, r)

#### 3. Creating Partition and Collection in Milvus

In [17]:
COLLECTION_NAME = 'demo_films'
PARTITION_NAME = 'Movie'

#Dropping collection for clean slate run
milv.drop_collection(COLLECTION_NAME)


param = {'collection_name':COLLECTION_NAME, 
         'dimension':32, 
         'index_file_size':2048, 
         'metric_type':MetricType.L2
        }

milv.create_collection(param)
milv.create_partition(COLLECTION_NAME, PARTITION_NAME)

Status(code=0, message='OK')

#### 4. Getting Embeddings and IDs
The vectors in `movie_vectors.txt` are obtained from the `user_vector_model` downloaded above. So we can directly get the vectors and the IDs by reading the file.

In [18]:
def get_vectors():
    with codecs.open("movie_recommender/movie_vectors.txt", "r", encoding='utf-8', errors='ignore') as f:
        lines = f.readlines()
    ids = [int(line.split(":")[0]) for line in lines]
    embeddings = []
    for line in lines:
        line = line.strip().split(":")[1][1:-1]
        str_nums = line.split(",")
        emb = [float(x) for x in str_nums]
        embeddings.append(emb)
    return ids, embeddings

ids, embeddings = get_vectors()

#### 4. Importing Vectors into Milvus
Import vectors into the partition **Movie** under the collection **demo_films**.

In [20]:
status = milv.insert(collection_name=COLLECTION_NAME, records=embeddings, ids=ids, partition_tag=PARTITION_NAME)
status[0]

Status(code=0, message='Add vectors successfully!')

### Recalling Vectors in Milvus
#### 1. Genarating User Embeddings
Pass in the gender, age and occupation of the user we want to recommend. **user_vector_model** model will generate the corresponding user vector.
Occupation is chosen from the following choices:
*  0:  "other" or not specified
*  1:  "academic/educator"
*  2:  "artist"
*  3:  "clerical/admin"
*  4:  "college/grad student"
*  5:  "customer service"
*  6:  "doctor/health care"
*  7:  "executive/managerial"
*  8:  "farmer"
*  9:  "homemaker"
*  10:  "K-12 student"
*  11:  "lawyer"
*  12:  "programmer"
*  13:  "retired"
*  14:  "sales/marketing"
*  15:  "scientist"
*  16:  "self-employed"
*  17:  "technician/engineer"
*  18:  "tradesman/craftsman"
*  19:  "unemployed"
*  20:  "writer"

In [21]:
import numpy as np
from paddle_serving_app.local_predict import LocalPredictor

class RecallServerServicer(object):
    def __init__(self):
        self.uv_client = LocalPredictor()
        self.uv_client.load_model_config("movie_recommender/user_vector_model/serving_server_dir") 
        
    def hash2(self, a):
        return hash(a) % 1000000

    def get_user_vector(self):
        dic = {"userid": [], "gender": [], "age": [], "occupation": []}
        lod = [0]
        dic["userid"].append(self.hash2('0'))
        dic["gender"].append(self.hash2('M'))
        dic["age"].append(self.hash2('23'))
        dic["occupation"].append(self.hash2('6'))
        lod.append(1)

        dic["userid.lod"] = lod
        dic["gender.lod"] = lod
        dic["age.lod"] = lod
        dic["occupation.lod"] = lod
        for key in dic:
            dic[key] = np.array(dic[key]).astype(np.int64).reshape(len(dic[key]),1)
        fetch_map = self.uv_client.predict(feed=dic, fetch=["save_infer_model/scale_0.tmp_1"], batch=True)
        return fetch_map["save_infer_model/scale_0.tmp_1"].tolist()[0]

recall = RecallServerServicer()
user_vector = recall.get_user_vector()

2021-05-18 13:06:14,131 - INFO - load_model_config params: model_path:movie_recommender/user_vector_model/serving_server_dir, use_gpu:False,            gpu_id:0, use_profile:False, thread_num:1, mem_optim:True, ir_optim:False,            use_trt:False, use_feed_fetch_ops:False


#### 2. Searching
Pass in the user vector, and then recall vectors in the previously imported data collection and partition.

In [22]:
TOP_K = 20
SEARCH_PARAM = {'nprobe': 20}
status, results = milv.search(collection_name=COLLECTION_NAME, query_records=[user_vector], top_k=TOP_K, params=SEARCH_PARAM)

#### 3. Returning Information by IDs

In [23]:
recall_results = []
for x in results[0]:
    recall_results.append(r.get("{}##movie_info".format(x.id)).decode('utf-8'))
recall_results

['{"movie_id": "760", "title": "Stalingrad (1993)", "genre": ["War"]}',
 '{"movie_id": "1350", "title": "Omen, The (1976)", "genre": ["Horror"]}',
 '{"movie_id": "1258", "title": "Shining, The (1980)", "genre": ["Horror"]}',
 '{"movie_id": "632", "title": "Land and Freedom (Tierra y libertad) (1995)", "genre": ["War"]}',
 '{"movie_id": "3007", "title": "American Movie (1999)", "genre": ["Documentary"]}',
 '{"movie_id": "2086", "title": "One Magic Christmas (1985)", "genre": ["Drama", "Fantasy"]}',
 '{"movie_id": "1051", "title": "Trees Lounge (1996)", "genre": ["Drama"]}',
 '{"movie_id": "3920", "title": "Faraway, So Close (In Weiter Ferne, So Nah!) (1993)", "genre": ["Drama", "Fantasy"]}',
 '{"movie_id": "1303", "title": "Man Who Would Be King, The (1975)", "genre": ["Adventure"]}',
 '{"movie_id": "652", "title": "301, 302 (1995)", "genre": ["Mystery"]}',
 '{"movie_id": "1605", "title": "Excess Baggage (1997)", "genre": ["Adventure", "Romance"]}',
 '{"movie_id": "1275", "title": "High

## Conclusion

After completing the recall service, the results can be further sorted using the **movie_recommender** model, and then the movies with high similarity scores can be recommended to users. You can try this deployable recommendation system using this [quick start](QUICK_START.md).